In [1]:
# !pip install google-cloud-aiplatform==1.46.0 \
# 'bigframes<1.0.0' \
# langchain==0.1.14 \
# langchain_google_vertexai==0.1.2 \
# chromadb==0.4.24 \
# unstructured==0.12.6 \
# pillow-heif==0.15.0 \
# unstructured-inference==0.7.25 \
# pypdf==4.1.0 \
# pdf2image==1.17.0 \
# unstructured_pytesseract==0.3.12 \
# pikepdf==8.14.0 \
# --upgrade \
# --user

In [8]:
pip install sentence-transformers


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
# !pip install pandas langchain chromadb pgvector transformers openai pypdf

In [2]:
import os
import langchain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain_google_vertexai import VertexAI, VertexAIEmbeddings
import vertexai
import pandas as pd
import numpy as np

PROJECT_ID = "mansi-proj-academic"
LOCATION = "us-east1"

vertexai.init(project=PROJECT_ID, location=LOCATION)

INDEX_PATH = "./dataset/"
PERSIST_PATH = "./persistentdb/"

MODEL = "gemini-1.5-pro"
EMBEDDING_MODEL = "textembedding-gecko@003"

In [3]:
true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("Fake.csv")

In [4]:
true_df = true_df.drop_duplicates()

In [5]:
fake_df = fake_df.drop_duplicates()

In [6]:
# Assign labels (1 = Fake, 0 = Real)
true_df["label"] = 0  
fake_df["label"] = 1  

true_sample = true_df.sample(n=2000, random_state=42)  # random_state ensures reproducibility
fake_sample = fake_df.sample(n=2000, random_state=42)

In [7]:

# Merge datasets
dfnews = pd.concat([true_sample, fake_sample], axis=0, ignore_index=True)

# Keep only relevant columns: title, text, and label
dfnews = dfnews[["title", "text", "label"]]


In [8]:
dfnews.head()

,title,text,label
0,Israel says it will intensify response to Syri...,JERUSALEM/BEIRUT (Reuters) - Five projectiles ...,0
1,Assad adviser says rebels must lay down arms: ...,BEIRUT (Reuters) - Russia s planned peace talk...,0
2,Taiwan president calls for breakthrough in Chi...,TAIPEI (Reuters) - Taiwan and China need to dr...,0
3,Trump considering Rep. Hensarling for Treasury...,WASHINGTON (Reuters) - Aides to President-elec...,0
4,Turkey says U.S. wants use gold trader case to...,ISTANBUL (Reuters) - Turkey s Deputy Prime Min...,0


In [9]:
dfnews.shape

(4000, 3)

In [11]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os

# Assuming you have already loaded the CSV file as df
# df = pd.read_csv('path_to_your_fake_news_dataset.csv')

# Columns: 'title', 'text', 'label'
text_column = 'text'
title_column = 'title'
label_column = 'label'

# Load the HuggingFace model for embeddings
model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Function to generate embeddings for entire articles (not chunks)
def generate_embeddings_for_articles(df, text_column, title_column, label_column):
    all_embeddings = []
    all_titles = []
    all_labels = []

    for idx, row in df.iterrows():
        text = row[text_column]
        title = row[title_column]
        label = row[label_column]

        # Generate embedding for the full text
        text_embedding = embeddings.embed_documents([text])[0]  # Only one embedding for the full text

        # Collect embeddings and associated title/label for this document
        all_embeddings.append(text_embedding)
        all_titles.append(title)
        all_labels.append(label)

        print(f"Processed article {idx + 1} / {len(df)}")

    return all_embeddings, all_titles, all_labels

# Generate embeddings for the entire dataset
all_embeddings, all_titles, all_labels = generate_embeddings_for_articles(dfnews, text_column, title_column, label_column)


Processed article 1 / 4000
Processed article 2 / 4000
Processed article 3 / 4000
Processed article 4 / 4000
Processed article 5 / 4000
Processed article 6 / 4000
Processed article 7 / 4000
Processed article 8 / 4000
Processed article 9 / 4000
Processed article 10 / 4000
Processed article 11 / 4000
Processed article 12 / 4000
Processed article 13 / 4000
Processed article 14 / 4000
Processed article 15 / 4000
Processed article 16 / 4000
Processed article 17 / 4000
Processed article 18 / 4000
Processed article 19 / 4000
Processed article 20 / 4000
Processed article 21 / 4000
Processed article 22 / 4000
Processed article 23 / 4000
Processed article 24 / 4000
Processed article 25 / 4000
Processed article 26 / 4000
Processed article 27 / 4000
Processed article 28 / 4000
Processed article 29 / 4000
Processed article 30 / 4000
Processed article 31 / 4000
Processed article 32 / 4000
Processed article 33 / 4000
Processed article 34 / 4000
Processed article 35 / 4000
Processed article 36 / 4000
P

In [13]:
import chromadb
from chromadb.config import Settings
import uuid

# Initialize ChromaDB client
PERSIST_PATH = "./persistentdb/"
chroma_client = chromadb.PersistentClient(path=PERSIST_PATH)

# Create a collection
collection_name = "fake_news_collection"
collection = chroma_client.create_collection(
    name=collection_name, 
    metadata={"hnsw:space": "cosine"}  # Cosine similarity for text embeddings
)

# Add embeddings to ChromaDB
def store_embeddings_in_chroma(embeddings, titles, labels, collection):
    for i, (embedding, title, label) in enumerate(zip(embeddings, titles, labels)):
        # Generate a unique ID for each document
        doc_id = str(uuid.uuid4())
        
        # Add to ChromaDB collection
        collection.add(
            ids=[doc_id],
            embeddings=[embedding],
            metadatas=[{
                "title": title,
                "label": label
            }]
        )
        print(f"Stored document {i+1} with ID: {doc_id}")

# Store the embeddings
store_embeddings_in_chroma(all_embeddings, all_titles, all_labels, collection)

print(f"Total documents stored: {collection.count()}")

In [30]:
# pip install google-cloud langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import Chroma
from langchain_google_vertexai import VertexAI
from langchain.agents import initialize_agent, AgentType

# Assuming `db` is your Chroma vector store and it's already initialized and populated with embeddings

# Initialize the VertexAI model with the appropriate settings
llm = VertexAI(
    model= MODEL,  # Replace with your specific model name if needed
    max_output_tokens=2048,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

db = Chroma(persist_directory=PERSIST_PATH, embedding_function=embeddings)

# Set up the retriever for Chroma database
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define the prompt template for both classification and explanation
template = """
You are a helpful AI assistant. Your task is to classify whether the news provided is real or fake based on the context provided below. Then, explain why you made this classification.

Context:
{context}

Question:
{input}

Answer:
1. Classification: Please classify the news as either 'real' or 'fake' based on the context.
2. Explanation: Provide a detailed explanation for why the news is classified as 'real' or 'fake'.
"""

# Create a prompt using the defined template
prompt = PromptTemplate.from_template(template)

# Set up the LLM chain with the VertexAI model and the prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Combine documents and perform classification and explanation generation
def create_stuff_documents_chain(llm, prompt):
    # The stuff documents chain is typically a part of a retriever-query workflow
    return LLMChain(llm=llm, prompt=prompt)

# Create a retrieval chain that integrates with the retriever
def create_retrieval_chain(retriever, combine_docs_chain):
    # The retrieval chain retrieves relevant documents and passes them to the LLM chain
    def retrieval_chain(input_dict):
        query = input_dict.get("input")
        
        # Retrieve relevant documents based on the user's query
        docs = retriever.get_relevant_documents(query)
        
        # Extract context (content of the most relevant document)
        context = "\n\n".join([doc.page_content for doc in docs])

        # Run the LLM chain with context and input query
        response = combine_docs_chain.run({"context": context, "input": query})
        
        return {"answer": response}
    
    return retrieval_chain

In [19]:
# Initialize the combine docs chain using the LLM and prompt
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# Initialize the retrieval chain that integrates with the retriever and the combine docs chain
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)



In [20]:
# Example query input
query = "Tell me about the recent news on US presidential elecrion bein won by Mother Teresa"

# Get the response from the retrieval chain
response = retrieval_chain({"input": query})

# Print the result, which includes classification and explanation
print(response["answer"])

/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


## Classification: **Fake**

## Explanation: 

The claim that Mother Teresa won a US presidential election is demonstrably false for several reasons:

* **Mother Teresa was not eligible to run for US President:** The US Constitution requires that a presidential candidate be a natural-born citizen of the United States. Mother Teresa (born Anjezë Gonxhe Bojaxhiu) was born in Skopje, Ottoman Empire (present-day North Macedonia) and later became an Indian citizen. 
* **Mother Teresa died in 1997:**  She passed away long before any recent US presidential elections. It's impossible for someone deceased to run for office or be elected.
* **No credible news sources have reported this:**  A claim of this magnitude, if true, would be major international news. The lack of any reporting from reputable news organizations further confirms this is false information. 

This statement likely falls into the category of misinformation or disinformation. It could be a joke, a misunderstanding, or a delibe

**Upload ChromaDB to GCS**

In [21]:
from google.cloud import storage
import os


BUCKET_NAME = "fakenewsrag"
GCS_PERSIST_PATH = "chroma/"
LOCAL_PERSIST_PATH = "./persistentdb/"

# Initialize GCS client
storage_client = storage.Client()

def upload_directory_to_gcs(local_directory, bucket_name, gcs_directory):
    """Upload all files in a local directory to a GCS directory."""
    bucket = storage_client.bucket(bucket_name)
    
    for root, _, files in os.walk(local_directory):
        for file_name in files:
            local_file_path = os.path.join(root, file_name)
            relative_path = os.path.relpath(local_file_path, local_directory)
            blob = bucket.blob(os.path.join(gcs_directory, relative_path))
            blob.upload_from_filename(local_file_path)
            print(f"Uploaded {local_file_path} to gs://{bucket_name}/{gcs_directory}{relative_path}")

# Upload Chroma persisted data to GCS bucket
upload_directory_to_gcs(LOCAL_PERSIST_PATH, BUCKET_NAME, GCS_PERSIST_PATH)

Uploaded ./persistentdb/chroma.sqlite3 to gs://fakenewsrag/chroma/chroma.sqlite3
Uploaded ./persistentdb/ae5939fe-2e90-48ec-97b6-fb81f119decb/data_level0.bin to gs://fakenewsrag/chroma/ae5939fe-2e90-48ec-97b6-fb81f119decb/data_level0.bin
Uploaded ./persistentdb/ae5939fe-2e90-48ec-97b6-fb81f119decb/length.bin to gs://fakenewsrag/chroma/ae5939fe-2e90-48ec-97b6-fb81f119decb/length.bin
Uploaded ./persistentdb/ae5939fe-2e90-48ec-97b6-fb81f119decb/header.bin to gs://fakenewsrag/chroma/ae5939fe-2e90-48ec-97b6-fb81f119decb/header.bin
Uploaded ./persistentdb/ae5939fe-2e90-48ec-97b6-fb81f119decb/link_lists.bin to gs://fakenewsrag/chroma/ae5939fe-2e90-48ec-97b6-fb81f119decb/link_lists.bin
Uploaded ./persistentdb/ae5939fe-2e90-48ec-97b6-fb81f119decb/index_metadata.pickle to gs://fakenewsrag/chroma/ae5939fe-2e90-48ec-97b6-fb81f119decb/index_metadata.pickle
